In [1]:
import torch
from torch import nn

from transformers import get_linear_schedule_with_warmup
from transformers import AdamW

from GetLoader import GetLoader
from MyDataset import MyDataset
from GetInit import GetInit
from TrainFunc import TrainFunc
from Bert import Bert

import warnings
warnings.filterwarnings("ignore")
torch.backends.cudnn.benchmark = True

In [2]:
import random
import numpy as np
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [3]:
data_root = {
    "train_path": '../../data/train_torch.csv',
    "test_path": "../../data/test_a.csv",
    "sub_path": "../../data/test_a_sample_submit.csv",
#     "w2v_path": "../../data/word2vec.bin",
    "bert_path": "./bert-mini/"
}
config = GetInit(data_root)

train_dataset = MyDataset(data_root["bert_path"],
                          corpus=config.x_train,
                          feature=config.x_train_feature,
                          corpus_label=config.y_train,
                          with_label=True)
test_dataset = MyDataset(data_root["bert_path"],
                         corpus=config.x_test,
                         feature=config.x_test_feature,
                         with_label=False)

loader = GetLoader(train_dataset, test_dataset)

GetInit Start!
GetInit End!
GetLoader End


In [4]:
# 建立model
model = Bert(bert_path=data_root["bert_path"], hidden_size=100, output_size=14, dropout=0.5)
model.cuda()
criterion = nn.NLLLoss()
opt = AdamW(model.parameters(),
            lr=5e-5,  # args.learning_rate - default is 5e-5
            eps=1e-8  # args.adam_epsilon  - default is 1e-8
            )
epochs = 2
total_steps = len(loader.train_loader) * epochs
scheduler = get_linear_schedule_with_warmup(opt,
                                            num_warmup_steps=0,
                                            num_training_steps=total_steps)

In [ ]:
# 开始训练
mytrain = TrainFunc(model, criterion, opt, scheduler, loader.train_loader, loader.valid_loader, loader.test_loader)
best_model = mytrain.train(epochs)

 
======== Epoch 1 / 2 ========
Training...
  Batch     0  of  22,500.  Loss:2.612825632095337      Elapsed: 0:00:00.
  Batch    80  of  22,500.  Loss:2.3251945972442627     Elapsed: 0:00:13.
  Batch   160  of  22,500.  Loss:1.902754783630371      Elapsed: 0:00:27.
  Batch   240  of  22,500.  Loss:2.0223584175109863     Elapsed: 0:00:41.
  Batch   320  of  22,500.  Loss:1.1111431121826172     Elapsed: 0:00:54.
  Batch   400  of  22,500.  Loss:1.3314710855484009     Elapsed: 0:01:06.
  Batch   480  of  22,500.  Loss:0.8565031886100769     Elapsed: 0:01:20.
  Batch   560  of  22,500.  Loss:0.8969120979309082     Elapsed: 0:01:33.
  Batch   640  of  22,500.  Loss:1.051419973373413      Elapsed: 0:01:45.
  Batch   720  of  22,500.  Loss:0.9986407160758972     Elapsed: 0:01:59.
  Batch   800  of  22,500.  Loss:0.7038463354110718     Elapsed: 0:02:17.
  Batch   880  of  22,500.  Loss:0.7338001728057861     Elapsed: 0:02:30.


In [ ]:
import pandas as pd
pd.set_option('precision', 2)
df_stats = pd.DataFrame(data=mytrain.training_stats)
df_stats = df_stats.set_index('epoch')
df_stats

In [ ]:
import pandas as pd
import numpy as np
ans=mytrain.predict()
df_sub=pd.read_csv(data_root["sub_path"])
df_sub.label=np.array(ans)
df_sub.label=df_sub.label.astype("int64")
save_name="./bertmini_{:.4f}.csv".format(mytrain.best_score)
df_sub.to_csv(save_name, index=False)